In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

Tackling Fashion MNIST with a CNN

数据预处理

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# ...表示np的索引
# np.newaxie表示新增维度
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

创建简单的CNN处理Fashion MNIST

In [5]:
from functools import partial
# 偏函数partial(原函数,原函数参数...)
# 将返回一个新的函数，该函数固定了原函数的某些参数
# partial用于增强原函数，简化使用
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation="relu", padding="SAME")
model = keras.models.Sequential([
    # 没有步幅是因为输入图像不大， 图象是28x28具有单个颜色通道
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    # 最大池化层，内核为2x2，步幅大小为2的池化层，最终将每个空间维度/2
    keras.layers.MaxPooling2D(pool_size=2),
    # 两个卷积层+一个最大池化层
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    # 随着CNN向输出层延申，滤波器的数量会增加，
    # 用更多不同方法将低层特征组成更高层次的特征
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    # 必须将输入展平，密集网络需要每个实例的一维特征阵列
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation="relu"),
    # dropout正则化，防止过拟合
    # https://blog.csdn.net/yangwohenmai1/article/details/123346240
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation="softmax"),
])

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10] # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10
1719/1719 [==============================] - 41s 20ms/step - loss: 0.6954 - accuracy: 0.7636 - val_loss: 0.3558 - val_accuracy: 0.8716
Epoch 2/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.4192 - accuracy: 0.8610 - val_loss: 0.3105 - val_accuracy: 0.8864
Epoch 3/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.3615 - accuracy: 0.8780 - val_loss: 0.2993 - val_accuracy: 0.8922
Epoch 4/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.3334 - accuracy: 0.8875 - val_loss: 0.2891 - val_accuracy: 0.8978
Epoch 5/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.3079 - accuracy: 0.8941 - val_loss: 0.2833 - val_accuracy: 0.8938
Epoch 6/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.2881 - accuracy: 0.9021 - val_loss: 0.3040 - val_accuracy: 0.9034
Epoch 7/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.2833 - accuracy: 0.9022 - val_loss: 0.2541 -